### Necessary imports

In [ ]:
!pip install -q -U torch datasets transformers tensorflow langchain playwright html2text sentence_transformers faiss-cpu
!pip install --upgrade -qr requirements.txt

### Dependencies

In [2]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)
from datasets import load_dataset
from peft import LoraConfig, PeftModel

from langchain.text_splitter import CharacterTextSplitter
from langchain.document_transformers import Html2TextTransformer
from langchain.document_loaders import AsyncChromiumLoader

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain

2024-02-25 23:00:04.881631: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 23:00:04.881687: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 23:00:04.882845: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-25 23:00:04.889738: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-25 23:00:06.047239: W tensorflow/compiler/tf2

### Load quantized Mistal 7B

In [2]:
#################################################################
# Tokenizer
#################################################################

model_name='mistralai/Mixtral-8x7B-Instruct-v0.1'

tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir='/raid/alex/cache', trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

#################################################################
# bitsandbytes parameters
#################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

#################################################################
# Load pre-trained config
#################################################################
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    cache_dir='/raid/alex/cache',
    revision="refs/pr/5",
    device_map='auto',
)

Your GPU supports bfloat16: accelerate training with bf16=True


Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


### Count number of trainable parameters

In [3]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 262410240
all model parameters: 23482601472
percentage of trainable model parameters: 1.12%


### Build Mistral text generation pipeline

In [4]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    use_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1000,
)

In [5]:
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

### Load and chunk documents. Load chunked documents into FAISS index 

In [8]:
import nest_asyncio
nest_asyncio.apply()

# Articles to index
articles = ["https://www.fantasypros.com/2023/11/rival-fantasy-nfl-week-10/",
            "https://www.fantasypros.com/2023/11/5-stats-to-know-before-setting-your-fantasy-lineup-week-10/",
            "https://www.fantasypros.com/2023/11/nfl-week-10-sleeper-picks-player-predictions-2023/",
            "https://www.fantasypros.com/2023/11/nfl-dfs-week-10-stacking-advice-picks-2023-fantasy-football/",
            "https://www.fantasypros.com/2023/11/players-to-buy-low-sell-high-trade-advice-2023-fantasy-football/"]

# Scrapes the blogs above
loader = AsyncChromiumLoader(articles)
docs = loader.load()

In [9]:
# Converts HTML to plain text 
html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)

# Chunk text
text_splitter = CharacterTextSplitter(chunk_size=100, 
                                      chunk_overlap=0)
chunked_documents = text_splitter.split_documents(docs_transformed)

# Load chunked documents into the FAISS index
db = FAISS.from_documents(chunked_documents, 
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

retriever = db.as_retriever()

Created a chunk of size 130, which is longer than the specified 100
Created a chunk of size 4706, which is longer than the specified 100
Created a chunk of size 131, which is longer than the specified 100
Created a chunk of size 230, which is longer than the specified 100
Created a chunk of size 500, which is longer than the specified 100
Created a chunk of size 207, which is longer than the specified 100
Created a chunk of size 365, which is longer than the specified 100
Created a chunk of size 312, which is longer than the specified 100
Created a chunk of size 515, which is longer than the specified 100
Created a chunk of size 584, which is longer than the specified 100
Created a chunk of size 1119, which is longer than the specified 100
Created a chunk of size 257, which is longer than the specified 100
Created a chunk of size 103, which is longer than the specified 100
Created a chunk of size 136, which is longer than the specified 100
Created a chunk of size 230, which is longer t

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [1]:
from langchain_community.document_loaders import UnstructuredFileLoader
from glob import glob
files = glob('/raid/alex/data/NatGeo/pdf/*pdf')
loader = UnstructuredFileLoader(files)
docs = loader.load()

1
PDF text extraction failed, skip text extraction...


TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.

In [ ]:
# Chunk text
text_splitter = CharacterTextSplitter(chunk_size=100, 
                                      chunk_overlap=0)
chunked_documents = text_splitter.split_documents(docs)

# Load chunked documents into the FAISS index
db = FAISS.from_documents(chunked_documents, 
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

retriever = db.as_retriever()

In [15]:
docs_transformed

[Document(page_content="By accessing this site you agree to our  \nprivacy policy and terms of use.  Accept\n\nSkip to main content\n\n__\n\nPremium __\n\n< Back Draft Wizard __\n\n< Back My Playbook __\n\n< Back NFL __\n\n< Back MLB __\n\n< Back NBA __\n\n< Back NHL __\n\n< Back DFS __\n\n< Back BettingPros\n\n< Back  More... __\n\n  *     * PGA\n\n< Back Draft Wizard __\n\n< Back My Playbook __\n\nGame DayLive!\n\nPremium\n\n__\n\n__\n\n__\n\nSearch for more!\n\nYou can now search for Tools, Reports, Articles, Rankings, as well as Players\nand Teams.\n\nStart New Search\n\nSign In\n\n__\n\n## Loading your leagues\n\n__\n\n  * My Account\n  * My Leagues\n  * Premium\n  * Mobile Apps\n  * Discord Chat\n  * Forum\n  * Sign In\n  * Sign Up\n  * Help\n  * Sign Out\n\nHelp\n\nNew\n\n  * NFL Home\n  * Tools __\n    *       * Draft\n      * Draft Simulator\n      * Mock Draft Lobby\n      * Live Draft Assistant\n      * Draft Kit\n      * Best Ball Draft Kit\n      * Cheat Sheet Creator\n   

### Create PromptTemplate and LLMChain

In [10]:
prompt_template = """
### [INST] Instruction: Answer the question based on your fantasy football knowledge. Here is context to help:

{context}

### QUESTION:
{question} [/INST]
 """

# Create prompt from prompt template 
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain 
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

In [11]:
llm_chain.invoke({"context": "", "question": "Should I start Gibbs in week 16 for fantasy?"})

/home/alex/.pyenv/versions/3.11.7/envs/rag/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'context': '',
 'question': 'Should I start Gibbs in week 16 for fantasy?',
 'text': "\nTo provide an accurate answer, I would need more information about the specific circumstances of your fantasy football league and roster. However, I can give you some general advice that might help you make a decision.\n\nFirstly, it's important to consider the matchup. Who is Gibbs playing against in week 16? If they have a strong defense or if Gibbs has struggled against them in the past, it might be worth considering other options.\n\nSecondly, what are the alternatives available on your bench or waiver wire? Are there any other running backs who have a better matchup or recent performance? It's important to weigh the potential upside of starting Gibbs against the potential downside of benching a player with higher upside.\n\nLastly, what are the playoff implications of this decision? Are you currently in a must-win situation or do you have some room for error? This can impact how aggressive or 

### Build RAG Chain

In [21]:
rag_chain = ( 
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

result = rag_chain.invoke("What can help me win in fantasy football?")

/home/alex/.pyenv/versions/3.11.7/envs/rag/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [22]:
result['context']

[Document(page_content='In this article, we’ll look at some standout statistics heading into Week 10\nand how they can impact your fantasy football team moving forward. Whether\nyou’re fighting for a playoff spot or looking to secure a bye week, the\ninsights and analysis in this article will help you make the right moves to\nwin your league.', metadata={'source': 'https://www.fantasypros.com/2023/11/5-stats-to-know-before-setting-your-fantasy-lineup-week-10/'}),
 Document(page_content='**Check out the rest of our weekly fantasy football advice**', metadata={'source': 'https://www.fantasypros.com/2023/11/players-to-buy-low-sell-high-trade-advice-2023-fantasy-football/'}),
 Document(page_content='Remember to read all the fantastic work here at FantasyPros and reach out on\nTwitter at @jpep20 if you need extra help. Good luck in Week 10!', metadata={'source': 'https://www.fantasypros.com/2023/11/5-stats-to-know-before-setting-your-fantasy-lineup-week-10/'}),
 Document(page_content='Week 

In [23]:
print(result['text'])


To increase your chances of winning in fantasy football, it's important to stay updated with the latest stats and trends, especially during the crucial stages of the season. With teams like the Kansas City Chiefs, Miami Dolphins, and Philadelphia Eagles on bye in Week 10, navigating the key stats becomes even more vital. By analyzing player and team performances, understanding matchups, and staying informed about injuries and other factors, you can make better decisions regarding your lineup and trades, ultimately helping you gain an edge over your opponents. You can also consider seeking expert advice from reputable sources such as FantasyPros, reading their articles, and engaging with them on Twitter for additional support.
